## Import modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
    
from sklearn.linear_model import RidgeCV # maybe used for GCV, as per the documentation

## Algorithm

**Input**: 

- S = Solution snaptshot matrix S (NxNs)
- e_s = Tolerance
- zeta = Correction factor
- param = Regularization parameter alpha* or parameter w
- u_ref = Reference solution

In [2]:
# Loads the FOM Snapshots as the matrix S
S = np.load('fom_snapshots.npy')

In [3]:
S.shape
# N = 9216
# Ns = 450

(9216, 450)

In [4]:
N = S.shape[0] 
Ns = S.shape[1]

In [5]:
e_s = 1e-6 # used 1e-6 to get a smaller n_tra. Tried 1e-7, but then Ns > n(n+1)/2 wouldn't verify

In [6]:
zeta = 0.15 # following the paper

In [7]:
omega = 0.3 # the paper uses 0.1. Changed to 0.3 to make some tests regarding alpha_star

In [8]:
u_ref = 0 # referece solution would be u(0) = 0

In [9]:
# Thin SVD of S
Us, SigmaS, YsT = np.linalg.svd(S, full_matrices=False)

In [10]:
Us.shape

(9216, 450)

In [11]:
SigmaS.shape

(450,)

In [12]:
YsT.shape

(450, 450)

In [13]:
# sets the initial value for sum_n = 0
sum_n = 0

# sets the value of sum_k as the sum of squared values of SigmaS vector
SigmaSSquared = np.square(SigmaS) 
sum_k = np.sum(SigmaSSquared)

print(sum_k)

2207187.5414926386


In [14]:
# Finds the minimum value of n that satisfies (4) 

for n in range (S.shape[1] - 1):
    sum_n += SigmaSSquared[n]
    if (sum_n/sum_k >= 1 - e_s):
        n_tra = n + 1
        break
        
print(n_tra)

31


In [15]:
# Create V, using only the first n_tra columns of US

V = Us[:, :n_tra]

In [16]:
# Determines n_qua
n_qua = ((9 + 8*n_tra)**(1/2) - 3)/2
n_qua = (1+zeta)*n_qua

In [17]:
n_qua

7.492951236581804

In [18]:
((1+8*Ns)**(1/2)-1)/2

29.504166377354995

In [19]:
# Determines n
n = min(n_qua, ((1+8*Ns)**(1/2)-1)/2)
n

7.492951236581804

In [20]:
# Rounds up n
n = int(np.ceil(n))
n

8

In [21]:
###############################################################

In [22]:
# Truncates V, s.t. V.shape = N x n 
# Maybe I should have used np.trunc ? Otherwise wouldn't make sense to have created V with n_tra columns earlier

V = V[:, :n]
V.shape

(9216, 8)

In [23]:
###############################################################

In [24]:
# Declare E, Q_ and H_

E = np.empty((N,Ns))
Q_ = np.empty((int(n*(n+1)/2), Ns))
H_ = np.empty((N,int(n*(n+1)/2)))

In [25]:
print(E.shape)
print(Q_.shape)
print(H_.shape)

(9216, 450)
(36, 450)
(9216, 36)


In [26]:
# Populates E and Q_

for i in range(Ns):
    qi = np.dot(V.T, S[:,i])
    E[:,i] = S[:,i] - np.dot(V,qi) - u_ref
    Q_[:,i] = np.unique(np.kron(qi,qi))

In [27]:
Q_.shape

(36, 450)

In [28]:
# Thin SVD of Q_

Uq_, Sigmaq_, Yq_T = np.linalg.svd(Q_, full_matrices=False)

In [29]:
# Checking if shapes have the correct shape
print(Uq_.shape)
print(Sigmaq_.shape)
print(Yq_T.shape)

(36, 36)
(36,)
(36, 450)


In [30]:
# alpha_star not specified, so:

In [31]:
# Declares alpha_best and an empry array of size N
alpha_best = np.empty(N)

In [32]:
# Calculates number of samples (n_samp) using omega and Nq. This is the number of alphas that will be part of the vector alpha_samp

Nq = Sigmaq_.shape[0]
print(Nq)
n_samp = int(np.ceil(omega*Nq))
n_samp

36


11

In [33]:
# Creates and populates alpha_samp

alpha_samp = np.geomspace(Sigmaq_[Nq-1], Sigmaq_[0], n_samp) # Changed logspace to geomspace so I could specify the boundaries using absolute number (logspace uses base10)

In [34]:
alpha_samp.shape

(11,)

In [35]:
print(Sigmaq_)

[2.43360465e+05 1.83643861e+04 7.49524657e+03 2.28668211e+03
 1.52314996e+03 1.23354500e+03 8.80564674e+02 4.83591050e+02
 4.28665915e+02 2.17796756e+02 1.53248340e+02 1.05293717e+02
 9.33636545e+01 7.01141870e+01 5.43861350e+01 4.88087662e+01
 4.23706733e+01 3.28692838e+01 1.91122467e+01 1.83321684e+01
 1.61272977e+01 1.22172030e+01 8.39559283e+00 5.88348071e+00
 5.34092165e+00 5.25593787e+00 4.10511202e+00 2.96334702e+00
 2.93541616e+00 2.34978575e+00 1.75694686e+00 1.43710252e+00
 1.29016821e+00 9.89040855e-01 8.53112032e-01 7.10198329e-01]


In [36]:
print(Sigmaq_[Nq-1])
print(Sigmaq_[0])

0.7101983294268509
243360.46549225601


In [37]:
print(alpha_samp)

[7.10198329e-01 2.54019282e+00 9.08560230e+00 3.24968122e+01
 1.16232559e+02 4.15733323e+02 1.48696886e+03 5.31849693e+03
 1.90228661e+04 6.80397939e+04 2.43360465e+05]


In [38]:
####################################################################

In [39]:
def ridge_cv(X, y, alphas):
    """
    Performs Ridge regression with cross-validation to find the optimal regularization
    parameter alpha using scikit-learn's RidgeCV method.
    
    Parameters:
    X (array-like, shape (n_samples, n_features)): Training data.
    y (array-like, shape (n_samples,)): Target values.
    alphas (array-like): List of regularization parameters to test.
    
    Returns:
    alpha_opt (float): Optimal regularization parameter.
    """
    ridge_cv = RidgeCV(alphas=alphas)
    ridge_cv.fit(X, y)
    alpha_opt = ridge_cv.alpha_
    
    return alpha_opt

In [40]:
####################################################################

In [41]:
# Uses GCV (ridge_cv) to determine the best regularization parameters

from scipy import stats
from IPython.display import clear_output

for i in range(N):
    print(i, "of", N) # Just to check how fast it is going
    G = np.empty(n_samp)
    for k in range(n_samp):
        G[k] = ridge_cv(Yq_T , Q_.T[k], alpha_samp) # Maybe Training/Target data are wrong
    alpha_best[i] = np.amin(G)
    clear_output(wait=True) # Clears output so only one line of jupyter lab is used to show progress status

###### old code ######
#values, counts = np.unique(alpha_best[i], return_counts=True)
#ind = np.argmax(counts)

#alpha_star = values[ind] # GCV function not implemented
###### old code ######


# Used mode to get the most common argument at the array
alpha_star = stats.mode(alpha_best)[0][0]

###### old code ######
#print(ind)
#print(values)
###### old code ######

print(alpha_star)

0.7101983294268509


/tmp/ipykernel_43596/3013548408.py:23: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  alpha_star = stats.mode(alpha_best)[0][0]


In [42]:
#######################################################################
''' Regarding the GCV funcion:

1. I tried calling it using (Us, S.T[k], alpha_samp), which is the data from the first SVD. This seems more accurate, since it uses the fom_snapshot as training data
2. It worked, but was way to expensive (~30% of the loop had been processede after about three hours)
3. Also, I was printing the results, and all lines of G were exactly the same, so something odd was happening here (maybe n_samp is too low?)
4. Switched the input data to (Yq_T, Q_.T[k], alpha_samp), which is the data from the second SVD. Now it runs way faster! Took about 15 minutes to run on my laptop.
5. Interestingly, the numerical values of G are the same that I was getting in the first run.
6. Before we can proceed to testing if H is indeed correct, I think it would be nice to check if the input data of the GCV function makes sense
7. If so, we are ready to test H
'''

' Regarding the GCV funcion:\n\n1. I tried calling it using (Us, S.T[k], alpha_samp), which is the data from the first SVD. This seems more accurate, since it uses the fom_snapshot as training data\n2. It worked, but was way to expensive (~30% of the loop had been processede after about three hours)\n3. Also, I was printing the results, and all lines of G were exactly the same, so something odd was happening here (maybe n_samp is too low?)\n4. Switched the input data to (Yq_T, Q_.T[k], alpha_samp), which is the data from the second SVD. Now it runs way faster! Took about 15 minutes to run on my laptop.\n5. Interestingly, the numerical values of G are the same that I was getting in the first run.\n6. Before we can proceed to testing if H is indeed correct, I think it would be nice to check if the input data of the GCV function makes sense\n7. If so, we are ready to test H\n'

In [43]:
# Calculates h_i and inputs it to H_, line by line

for i in range(N):
    h_i = 0
    for l in range(Nq):
        h_i += ((Sigmaq_[l]**2)/(Sigmaq_[l]**2 + alpha_star**2))*np.dot((np.dot(Yq_T[l],E[l,:].T)/Sigmaq_[l]),Uq_[l])
    H_[i,:] = h_i
    
print(H_.shape)
print(h_i.shape)
    

(9216, 36)
(36,)


In [44]:
print(H_) # all lines are the same, ?

[[ 0.00142634  0.00257235 -0.00029587 ... -0.00049298  0.00067194
  -0.00038014]
 [ 0.00142634  0.00257235 -0.00029587 ... -0.00049298  0.00067194
  -0.00038014]
 [ 0.00142634  0.00257235 -0.00029587 ... -0.00049298  0.00067194
  -0.00038014]
 ...
 [ 0.00142634  0.00257235 -0.00029587 ... -0.00049298  0.00067194
  -0.00038014]
 [ 0.00142634  0.00257235 -0.00029587 ... -0.00049298  0.00067194
  -0.00038014]
 [ 0.00142634  0.00257235 -0.00029587 ... -0.00049298  0.00067194
  -0.00038014]]


In [47]:
# Recomposing S

In [46]:
'''
S = [s_1,s_2,...,s_n]
S=U S V^T
s_1 \approx U@U^T@s_1
q_1 = U^T@s_1
such that:
s_1 \approx H(q_1\kron q_1) + Uq_1
'''

'\nS = [s_1,s_2,...,s_n]\nS=U S V^T\ns_1 \x07pprox U@U^T@s_1\nq_1 = U^T@s_1\nsuch that:\ns_1 \x07pprox H(q_1\\kron q_1) + Uq_1\n'